<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"
alt="Rubin Observatory logo, a graphical representation of turning stars into data."> <br><br>
**Description:** Generate parameters for the _Rubin Observatory DP1_ paper. <br>
**Authors:** Leanne Guy, James Mullaney <br>
**Last verified to run:** 2025-05-14 <br>
**LSST Science Pipelines version:** v29.0.0<br>
**Container size:** medium <br>

# DP1 parameters

## Setup

In [3]:
! pip install inflect > /dev/null 2>&1
! pip install haversine > /dev/null 2>&1

In [2]:
# Generic python packages
import os
import csv
import requests
import yaml
import numpy as np
import pandas as pd
import numpy as np
from pathlib import Path
from haversine import haversine, Unit

# LSST Science Pipelines 
from lsst.daf.butler import Butler

# RSP
from lsst.rsp import get_tap_service

import warnings
warnings.filterwarnings("ignore")

## Define utility functions

In [2]:
bands = ['u', 'g', 'r', 'i', 'z', 'y']

In [3]:
# Create a dictionary to convert numbers to words.
# Used for those numbers best expressed in words.
def num2word(num):
    '''
    Converts select integer values into their word equivalent.
    If the integer is not one of the select values, then the integer
    is returned as a string to ensure consistent return type.
    '''
    num2wordDict = {1: 'One', 2: 'Two', 3: 'Three', 4: 'Four', 5: 'Five', \
                    6: 'Six', 7: 'Seven', 8: 'Eight', 9: 'Nine', 10: 'Ten', \
                    11: 'Eleven', 12: 'Twelve', 13: 'Thirteen', 14: 'Fourteen', \
                    15: 'Fifteen', 16: 'Sixteen', 17: 'Seventeen', 18: 'Eighteen', \
                    19: 'Nineteen', 20: 'Twenty', 30: 'Thirty', 40: 'Forty', \
                    50: 'Fifty', 60: 'Sixty', 70: 'Seventy', 80: 'Eighty', \
                    90: 'Ninety', 0: 'Zero'}
    if num in num2wordDict:
        return num2wordDict[num]
    else:
        return str(num)

In [4]:
# Function to round to N significant figures
def round_sf(x, sig=3):
    return np.round(x, sig - int(np.floor(np.log10(abs(x)))) - 1)

In [5]:
def formatParameter(params, name):
    '''
    Formats a named parameter ready for write-out to tex file.
    '''

    value = params[0][name]

    if isinstance(value, float):
        value = f'{value:.3f}'.rstrip('0').rstrip('.')

    if name in params[1]:
        unit = params[1][name]
        if unit not in ['\\arcsec', '\\arcmin', '\\degree']:
            unit = f'\\xspace {unit}'
    else:
        unit = ''

    return f'\\newcommand{{\\{name}}}{{{value}{unit}\\xspace}}\n'

In [6]:
def addParameter(params, name, value, unit=None, **kwargs):

    '''
    Adds a named parameter to the params tuple of dictionaries.

    Ensures that the namings between the values and units dictionaries are consistent.
    '''

    if 'sig' in kwargs:
        value = round_sf(value, **kwargs)

    params[0][name] = value
    if unit:
        params[1][name] = unit

    return params

In [7]:
def imageStats(params, imageType):
    '''
    Given an image dataset type name, add the following to the params:
    - Number of dataset types in DP1;
    - HDD size of the first instance of the dataset type in MB;
    - Number of pixels in each x/y dimension;
    - Platescale of image;
    - On-sky field of view of image, both in x/y and area.
    '''

    imageName = imageType.replace('_', '')

    # Number of images:
    refs = list(registry.queryDatasets(imageType))
    params = addParameter(params, f'n{imageName}s', len(list(refs)))

    # Image HDD size:
    filepath = butler.getURI(refs[0])
    roughFileSize = round_sf(os.path.getsize(filepath.path) / 1e6, sig=2)
    params = addParameter(params, f'{imageName}hdd', f'{roughFileSize:.0f}', unit='MB')

    # Number of pixels
    image = butler.get(refs[0])
    params = addParameter(params, f'n{imageName}pixx', image.getDimensions().x)
    params = addParameter(params, f'n{imageName}pixy', image.getDimensions().y)

    # Platescale:
    platescale = image.getWcs().getPixelScale().asArcseconds()
    params = addParameter(params, f'{imageName}platescale', f'{platescale:.1f}', unit='arcsec per pixel')

    # Field of view
    fovx = image.getDimensions().x * image.getWcs().getPixelScale().asDegrees()
    fovy = image.getDimensions().y * image.getWcs().getPixelScale().asDegrees()
    params = addParameter(params, f'{imageName}fovx', f'{fovx:.2f}', unit='\\degree')
    params = addParameter(params, f'{imageName}fovy', f'{fovy:.2f}', unit='\\degree')
    area = fovx * fovy
    roughArea = round_sf(area, sig=3)
    params = addParameter(params, f'{imageName}fov', f'{roughArea:.3f}', unit='deg$^2$')

    return params

## Set up DP1 Butler

In [8]:
instrument = 'LSSTComCam'
collections = ['LSSTComCam/DP1/defaults', 
               'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260',
               'skymaps', ]
skymapName = 'lsst_cells_v1'
butler = Butler("/repo/dp1",
                instrument=instrument, 
                collections=collections, 
                skymap=skymapName)
registry = butler.registry
skymap = butler.get('skyMap', skymap=skymapName)

In [9]:
## Consdb
cdb = get_tap_service("consdbtap")
assert cdb is not None

## Set up parameters dictionary

In [10]:
# Use a tuple of Python dicts to hold parameter values and units.
# The first dict in the tuple holds values, the second holds units.

# The dictionary keys are the names of the parameters in the parameter.tex files,
# but without the leading backslash.

# Pass the params tuple to the addParameter function to add a parameter value
# with an optional unit.
params = (dict(), dict())

## Add the parameters in the manualParameters.csv file.
There are some parameters that either can't be generated automatically, or take a long time to compute. Such parameters are contained in the `../manualParameters.csv` file, which is read-in using the following block. 

In the case of the those parameters that take a long time to compute, the code blocks to generate them can be found toward the bottom of this notebook. To re-calculate these parameters, the corresponding line should be removed from the `../manualParameters.csv` file. 


In [11]:
with open('../data/manualParameters.csv', newline='') as f:
    reader = csv.reader(f, delimiter=',')
    for line in reader:
        if line[2] == '':
            line[2]=None
        params = addParameter(params, line[0], line[1], unit=line[2])

## Compute parameters

### Information about the observing campaign:

In [12]:
# Number of visits:
visitRecords = registry.queryDimensionRecords('visit')
params = addParameter(params, 'nvisits', len(list(visitRecords)))

# Number of exposures. This should be the same as the number of visits.
exposureRecords = registry.queryDimensionRecords('exposure')
params = addParameter(params, 'nexposures', len(list(visitRecords)))               

# Number of target fields:
# Note: slew_icrs covered ECDFS, so they are the same field:
fields = set([record.target_name for record in exposureRecords if record.target_name != 'slew_icrs'])
params = addParameter(params, 'nfields', num2word(len(fields)).lower())

# First and last date of DP1 observations:
visit_table = butler.get('visit_table')
firstVisitTime = min(visit_table['obsStart'])
lastVisitTime = max(visit_table['obsStart'])
lastVisitNight = lastVisitTime - np.timedelta64(1,'D') # The last night started the date before the last observation datetime. 
params = addParameter(params, 'dponestartdate', np.datetime_as_string(firstVisitTime, unit='D'))
params = addParameter(params, 'dponeenddate', np.datetime_as_string(lastVisitNight, unit='D'))

# Median exposure time:
params = addParameter(params, 'exposuretime', f'{np.median(visit_table['expTime']):.0f}', unit='s')

### Observation quality

In [13]:
# Best seeing:
visit_detector_table = butler.get('visit_detector_table')
minSeeing = np.min(visit_detector_table['seeing'][visit_detector_table['nPsfStar']>100])
params = addParameter(params, 'bestimagequality', minSeeing, unit='\\arcsec', sig=2)

# Median seeing:
medSeeing = np.median(visit_detector_table['seeing'][visit_detector_table['nPsfStar']>100])
params = addParameter(params, 'medianimagequalityallbands', medSeeing, unit='\\arcsec', sig=3)

### Statistics for different types of image dataset

In [14]:
datasetNames = ['raw',
                'visit_image',
                'deep_coadd',
                'template_coadd',
                'difference_image',
               ]
for datasetName in datasetNames:
    params = imageStats(params, datasetName)

# The total number of pixels in a single deep_coadd image:
params = addParameter(params, 'ndeepcoaddpixtotal',
                      params[0]['ndeepcoaddpixx']*params[0]['ndeepcoaddpixy']/1e6,
                      sig=3, unit='million')

### Skymap data 

In [15]:
# Total number of tracts across the entire sky:
params = addParameter(params, 'ntotaltracts', len(skymap))

# The number of tracts covered by DP1:
tractRecords = list(registry.queryDimensionRecords('tract', where='visit > 0'))
tractIds = set([record.id for record in tractRecords])
params = addParameter(params, 'ntracts', len(tractIds))
params = addParameter(params, 'ncoveredtracts', len(tractIds))

# Area of each tract:
tract = skymap.generateTract(9000)
verticesInDegrees = [(vertex[0].asDegrees()%360 - 180, vertex[1].asDegrees()) for vertex in tract.getVertexList()]
tractArea = (
    haversine(verticesInDegrees[0][::-1], verticesInDegrees[1][::-1], unit=Unit.DEGREES) * 
    haversine(verticesInDegrees[1][::-1], verticesInDegrees[2][::-1], unit=Unit.DEGREES)
)
roughTractArea = round_sf(tractArea, sig=2)
params = addParameter(params, 'tractarea', f'{roughTractArea:.1f}', unit='deg$^2$')

# Number of patches:
numXPatches, numYPatches = skymap[0].getNumPatches()
numPatches = numXPatches * numYPatches
params = addParameter(params, 'npatchx', numXPatches)
params = addParameter(params, 'npatchy', numYPatches)
params = addParameter(params, 'npatch', numPatches)

# Area of each patch:
patchArea = (tractArea / numPatches)
roughPatchArea = round_sf(patchArea, sig=2)
params = addParameter(params, 'innerpatcharea', f'{roughPatchArea:.3f}', unit='')

refs = list(registry.queryDatasets('deep_coadd'))
coadd = butler.get(refs[0])
fovx = coadd.getDimensions().x * coadd.getWcs().getPixelScale().asDegrees()
fovy = coadd.getDimensions().y * coadd.getWcs().getPixelScale().asDegrees()
roughOuterArea = round_sf(fovx * fovy, sig=2)
params = addParameter(params, 'outerpatcharea', f'{roughOuterArea:.3f}', unit='deg$^2$')

### Exposure Time

In [ ]:
refs = list(registry.queryDatasets('visit_table'))
result = df_visits.groupby(['band', 'expTime']).size().unstack(fill_value=0).reindex(bands)
params = addParameter(params, 'exposuretime', )
params = addParameter(params, 'exposuretimeuband', )



df_visits = visits.to_pandas()


result

### Visit image selection criteria for incorporation into deep coadd

In [16]:
# The seeing criterion is communicated in the config file: 
refs = list(registry.queryDatasets('selectDeepCoaddVisits_config'))
config = butler.get(refs[0])
params = addParameter(params, 'deepcoaddmaxfwhm', config.maxPsfFwhm, sig=2, unit='\\arcsec')

## Survey property maps

In [17]:
# Find all HealSparseMap dataset types matching a pattern
allSuveyPropMaps = []
for datasetType in butler.registry.queryDatasetTypes():
    if registry.queryDatasets(datasetType).any(
        execute=False, exact=False):
        if (datasetType.storageClass.name == "HealSparseMap"):
            allSuveyPropMaps.append(datasetType.name)
            #print(f"Name: {datasetType.name}, Dimensions: {datasetType.dimensions}, Storage Classs: {datasetType.storageClass}")
print(f"Total of {len(allSuveyPropMaps)} survey properyy maps in DP1 ")
params = addParameter(params, 'nsurveypropertymaps',len(allSuveyPropMaps))

Total of 29 survey properyy maps in DP1 


### The total number of visit_images that went into coadds
In other words, the number that passed the various selection criteria:

In [18]:
#allVisitImages = []
#dataIds = registry.queryDataIds(['tract','patch','band','skymap'], datasets='deep_coadd')
#for dataId in dataIds:
#    inputs = butler.get('deep_coadd.coaddInputs', dataId=dataId)
#    allVisitImages.extend([row['id'] for row in inputs.ccds])

### The number of datasets of each type of catalog 

In [17]:
refs = list(registry.queryDatasets('source'))
params = addParameter(params, 'nsourcecatalogs', num2word(len(refs)).lower())

refs = list(registry.queryDatasets('object'))
params = addParameter(params, 'nobjectcatalogs', num2word(len(refs)).lower())

refs = list(registry.queryDatasets('dia_object'))
params = addParameter(params, 'ndiaobjectcatalogs', num2word(len(refs)).lower())

refs = list(registry.queryDatasets('dia_source'))
params = addParameter(params, 'ndiasourcecatalogs', num2word(len(refs)).lower())

refs = list(registry.queryDatasets('ss_source'))
params = addParameter(params, 'nsolarsystemsourcecatalogs', num2word(len(refs)).lower())

refs = list(registry.queryDatasets('visit_table'))
params = addParameter(params, 'nvisitsummarytables', num2word(len(refs)).lower())

refs = list(registry.queryDatasets('visit_detector_table'))
params = addParameter(params, 'nvisitdetectorsummarytables', num2word(len(refs)).lower())

refs = list(registry.queryDatasets('object_forced_source'))
params = addParameter(params, 'nobjectforcedcatalogs', num2word(len(refs)).lower())

refs = list(registry.queryDatasets('dia_object_forced_source'))
params = addParameter(params, 'ndiaobjectforcedcatalogs', num2word(len(refs)).lower())

### The number of entries in the visit, visit detector, and solar system source tables 

In [18]:
# There is only one of each of these dataset types:
refs = list(registry.queryDatasets('visit_table'))
table = butler.get(refs[0])
params = addParameter(params, 'nvisitsummaries', len(table))

refs = list(registry.queryDatasets('visit_detector_table'))
table = butler.get(refs[0])
params = addParameter(params, 'nvisitdetectorsummaries', len(table))

refs = list(registry.queryDatasets('ss_source'))
catalog = butler.get(refs[0]) # There is only 1 ss_source catalog in DP1.
params = addParameter(params, 'nsolarsystemsources', len(catalog))

refs = list(registry.queryDatasets('ss_object'))
catalog = butler.get(refs[0])
params = addParameter(params, 'nsolarsystemobjects', len(catalog))

In [19]:
refs = list(registry.queryDatasets('ss_source'))
catalog = butler.get(refs[0]) # There is only 1 ss_source catalog in DP1.

### Paramters used in the paper but not yet extracted programatically

In [38]:
params = addParameter(params, 'nnewasteroiddiscoveries', 93)
params = addParameter(params, 'nexposuresaoscommissioning',10,000)
params = addParameter(params, 'nexposurescalibcommissioning',2,000)
params = addParameter(params, 'exposuresspcommissioning',2,000)
params = addParameter(params, 'sciencepipelinesversion', 'v29.1')
params = addParameter(params, 'sciencepipelinesurl', 'https://pipelines.lsst.io/v/v29_1_1')

### Miscellaneous

#### The number of raws that failed processing to visit images

In [20]:
# Report the number of failed raw-to-visit_image processings as the difference in the
# number of raws and the number of visit_images.
params = addParameter(params, 'nsfpfails', params[0]['nraws'] - params[0]['nvisitimages'])

## Longer calculations
The following code blocks take more than a few seconds to run, in some cases many minutes. They run only if the respective parameter does not already exist in the params tuple, i.e., they have been removed from the "add by hand" block above. 

### Total area covered by DP1

In [21]:
# Takes around 10 minutes to run:
if 'totalarea' not in params[0]:
    coaddRefs = list(registry.queryDatasets('deep_coadd', where="band = 'r'"))
    pixcount = 0
    for coaddRef in coaddRefs:
        im = butler.get('deep_coadd.mask', dataId = coaddRef.dataId)
        pixcount +=  im.array.size - np.sum((im.array & im.getPlaneBitMask('NO_DATA')) > 0)
    area = pixcount * ((0.2 / 3600)**2)

    params = addParameter('totalarea', f'{np.round(area):.0f}', unit='deg$^2$')

### Total number of objects

In [22]:
# Takes around 8s to run:
if 'nobjects' not in params[0]:

    runningTotal = 0
    refs = list(registry.queryDatasets('object'))
    for ref in refs:
        catalog = butler.get(ref, parameters = {'columns':'objectId'})
        runningTotal += len(catalog)

    params = addParameter(params, 'nobjects', runningTotal/1e6, sig=2, unit='million')

### Total number of sources

In [23]:
# Takes around 2m to run:
if 'nsources' not in params[0]:

    refs = list(registry.queryDatasets('source'))
    runningTotal = 0
    for ref in refs:
        catalog = butler.get(ref, parameters = {'columns':'sourceId'})
        runningTotal += len(catalog)

    params = addParameter(params, 'nsources', runningTotal/1e6, sig=2, unit='million')

### Total number of DIA objects

In [24]:
# Takes less than a second, but placed here to group with similar tasks:
if 'ndiaobjects' not in params[0]:

    refs = list(registry.queryDatasets('dia_object'))
    runningTotal = 0
    for ref in refs:
        catalog = butler.get(ref, parameters = {'columns':'diaObjectId'})
        runningTotal += len(catalog)

    params = addParameter(params, 'ndiaobjects', runningTotal/1e6, sig=2, unit='million')

### Total number of DIA sources

In [25]:
# Takes less than a second, but placed here to group with similar tasks:
if 'ndiasources' not in params[0]:

    refs = list(registry.queryDatasets('dia_source'))
    runningTotal = 0
    for ref in refs:
        catalog = butler.get(ref, parameters = {'columns':'diaSourceId'})
        runningTotal += len(catalog)

    params = addParameter(params, 'ndiasources', runningTotal/1e6, sig=2, unit='million')

### Total number of forced objects and sources

In [26]:
# Takes around 15s to run:
if 'nforcedsources' not in params[0]:

    refs = list(registry.queryDatasets('object_forced_source'))
    runningTotalSrc = 0
    runningTotalObj = 0
    for ref in refs:
        catalog = butler.get(ref, parameters = {'columns':'objectId'})
        runningTotalSrc += len(catalog)
        runningTotalObj += len(catalog.to_pandas()['objectId'].unique())

    params = addParameter(params, 'nforcedsources', runningTotalSrc/1e6, sig=3, unit='million')
    params = addParameter(params, 'nforcedobjects', runningTotalObj/1e6, sig=2, unit='million')

### Total number of DIA forced objects and sources

In [27]:
# Takes around 1m30s to run:
if 'ndiaforcedsources' not in params[0]:

    refs = list(registry.queryDatasets('dia_object_forced_source'))
    runningTotalSrc = 0
    runningTotalObj = 0
    for ref in refs:
        catalog = butler.get(ref, parameters = {'columns':'diaObjectId'})
        runningTotalSrc += len(catalog)
        runningTotalObj += len(catalog.to_pandas()['diaObjectId'].unique())

    params = addParameter(params, 'ndiaforcedsources', runningTotalSrc/1e6, sig=3, unit='million')
    params = addParameter(params, 'ndiaforcedobjects', runningTotalObj/1e6, sig=2, unit='million')

### Total number of extended objects

In [28]:
# Takes around 10s to run:
if 'nextendedobjects' not in params[0].keys():

    nGals = 0
    nStars = 0
    nAll = 0 

    refs = list(registry.queryDatasets('object'))
    columns = [band + '_extendedness' for band in ['u', 'g', 'r', 'i', 'z', 'y']]
    for ref in refs:
        objectTable = butler.get(ref, parameters = {'columns':columns})
        if len(objectTable) > 0:
            galSelection = (
                ((objectTable['u_extendedness'] > 0.5) & ~objectTable['u_extendedness'].mask) |
                ((objectTable['g_extendedness'] > 0.5) & ~objectTable['g_extendedness'].mask)  |
                ((objectTable['r_extendedness'] > 0.5) & ~objectTable['r_extendedness'].mask)  |
                ((objectTable['i_extendedness'] > 0.5) & ~objectTable['i_extendedness'].mask)  |
                ((objectTable['z_extendedness'] > 0.5) & ~objectTable['z_extendedness'].mask)  |
                ((objectTable['y_extendedness'] > 0.5) & ~objectTable['y_extendedness'].mask) 
            )
            nGals += np.sum(galSelection.data)

            starSelection = (
                ((objectTable['u_extendedness'] <= 0.5) & ~objectTable['u_extendedness'].mask) &
                ((objectTable['g_extendedness'] <= 0.5) & ~objectTable['g_extendedness'].mask)  &
                ((objectTable['r_extendedness'] <= 0.5) & ~objectTable['r_extendedness'].mask)  &
                ((objectTable['i_extendedness'] <= 0.5) & ~objectTable['i_extendedness'].mask)  &
                ((objectTable['z_extendedness'] <= 0.5) & ~objectTable['z_extendedness'].mask)  &
                ((objectTable['y_extendedness'] <= 0.5) & ~objectTable['y_extendedness'].mask) 
            )
            nStars += np.sum(starSelection.data)

    params = addParameter(params, 'nextendedobjects', nGals/1e6, sig=2, unit='million')

### The total number of visit_images that went into coadds
In other words, the number that passed the various selection criteria:

In [29]:
# Takes around 5m to run:
# deep_coadd_input_map isn't available in the public data release.
if 'ndeepcoaddvisitimages' not in params[0].keys():
    refs = list(registry.queryDatasets('deep_coadd_input_map'))  

    visitDetectorPairs = set()
    for ref in refs:
        inputMap = butler.get(ref)
        keyRoots = [key[:5] for key in inputMap.metadata if key.startswith("B") & key.endswith('CCD')]
        visitDetectorPairs.update({(inputMap.metadata[keyRoot+'VIS'],inputMap.metadata[keyRoot+'CCD']) for keyRoot in keyRoots})
    params = addParameter(params, 'ndeepcoaddvisitimages', len(visitDetectorPairs))

In [31]:
# Takes around 1m30s to run.

# This method is not as robust as the above (it assumes all visit_images from a selected visit are used),
# but the equivalent of `deep_coadd_input_map` doesn't exist for template coadds.

# template_coadd_visit_selection isn't available in the public data release.
if 'ntemplatecoaddvisitimages' not in params[0].keys():
    refs = list(registry.queryDatasets('template_coadd_visit_selection'))

    visits = set()
    for ref in refs:
        inputVisits = butler.get(ref)
        visits.update({key for (key,value) in inputVisits.items() if value})

    nTemplateCoaddInputs = 0
    for visit in visits:
        nTemplateCoaddInputs += len(list(registry.queryDatasets('visit_image', visit=visit, instrument='LSSTComCam')))
    params = addParameter(params, 'ntemplatecoaddvisitimages', nTemplateCoaddInputs)

## Write the parameters to file

In [32]:
with open("../parameters.tex", "w") as f:
    f.write('% These parameters are automatically generated by the dp1_parameters notebook.\n')
    f.write('% Do NOT manually edit this file.\n')
    f.write('% If you need to change/add a parameter, please edit the dp1_parameters notebook and re-run it.\n\n')
    for name in params[0]:
        f.write(formatParameter(params, name))
f.close()